In [14]:
from data_pre_processing import *
from pair_finder import *

import statsmodels.tsa.stattools as ts

# Get Data 
---

In [15]:
prices = pd.read_csv("data/prices_data.csv", index_col=0, parse_dates=True)


In [16]:
prices

,BTC/USDT,ETH/USDT,XRP/USDT,LTC/USDT,BCH/USDT
timestamp,,,,,
2024-02-23,50744.15,2922.24,0.5342,68.80,264.7
2024-02-24,51568.22,2992.62,0.5450,70.41,267.7
2024-02-25,51728.85,3112.59,0.5425,70.16,267.9
2024-02-26,54476.47,3175.94,0.5506,71.93,274.7
2024-02-27,57037.34,3242.36,0.5861,73.97,292.8
...,...,...,...,...,...
2025-02-17,95780.00,2744.05,2.6599,122.31,323.8
2025-02-18,95671.74,2671.99,2.5628,129.75,316.3
2025-02-19,96644.37,2715.50,2.7378,134.88,322.8


# Applly funnel and test for cointegration
---

In [17]:

# Pre-filter: Calculate correlation matrix and select pairs with high correlation
corr_threshold = 0.8
corr_matrix, high_corr_pairs = filter_high_correlation_pairs(prices, threshold=corr_threshold)
print(f"\nPairs with correlation >= {corr_threshold}:")
for pair in high_corr_pairs:
    print(f"{pair[0]} & {pair[1]}: correlation = {pair[2]:.4f}")

# Next, test for cointegration on the pre-filtered pairs
cointegrated_pairs = []
for sym1, sym2, corr_val in high_corr_pairs:
    S1 = prices[sym1]
    S2 = prices[sym2]
    score, pvalue, _ = ts.coint(S1, S2)
    if pvalue < 0.05:
        cointegrated_pairs.append((sym1, sym2, pvalue, corr_val))

if cointegrated_pairs:
    print("\nCointegrated pairs (from pre-filtered high-correlation pairs):")
    for pair in cointegrated_pairs:
        print(f"{pair[0]} & {pair[1]}: p-value = {pair[2]:.4f}, correlation = {pair[3]:.4f}")
else:
    print("\nNo cointegrated pairs found among the high-correlation pairs.")


Pairs with correlation >= 0.8:
BTC/USDT & XRP/USDT: correlation = 0.9238
BTC/USDT & LTC/USDT: correlation = 0.8630
XRP/USDT & LTC/USDT: correlation = 0.8555

Cointegrated pairs (from pre-filtered high-correlation pairs):
BTC/USDT & XRP/USDT: p-value = 0.0333, correlation = 0.9238
